# 07 - Manhattan Subway Ridership Model Development  
Analysis-Driven Machine Learning Pipeline with Business Logic Validation

---

## Notebook Overview

**Objective:**  
Develop production-ready machine learning models that capture and validate discovered ridership behavior patterns.

**Input Dataset:**  
- `subway_ridership_modeling_features.parquet`  
- 24 validated features from prior feature engineering phase

**Output Artifacts:**  
- Trained and validated model object (e.g. XGBoost)
- Evaluation metrics and comparison summary
- Feature importance analysis
- Business logic validation of key model outputs

**Modeling Focus Areas:**
- Pattern fidelity (rush hour, weekend, weather responsiveness)
- Predictive accuracy across time (Jan–Oct train / Nov–Dec test)
- Interpretability and deployment readiness

---

In [1]:
# =============================================
# Setup and Configuration
# =============================================

import pandas as pd
import numpy as np
import json
import joblib
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Core ML
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Optional Boosting Models
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False

# Reproducibility
np.random.seed(42)

# NOTE: np.bool_, np.integer, np.floating will need conversion before JSON serialization
# See: convert_to_json_serializable() helper later in the pipeline

# Notebook header
print("\n" + "=" * 60)
print("NOTEBOOK 07: MANHATTAN SUBWAY MODEL DEVELOPMENT")
print("=" * 60)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Methodology: Model Comparison + XGBoost Optimization")
print("=" * 60)



NOTEBOOK 07: MANHATTAN SUBWAY MODEL DEVELOPMENT
Analysis Date: 2025-07-28 20:31:10
Methodology: Model Comparison + XGBoost Optimization


In [2]:
# =============================================
# 1. Data Loading and Temporal Splitting
# =============================================

def load_and_prepare_data():
    """Load dataset and separate features, target, and identifiers."""
    print("\n1. DATA LOADING AND PREPARATION")
    print("-" * 40)

    # Look for dataset
    possible_paths = [
        "../data/processed/modeling/subway_ridership_modeling_features.parquet",
        "data/processed/modeling/subway_ridership_modeling_features.parquet"
    ]

    dataset_path = next((Path(p) for p in possible_paths if Path(p).exists()), None)
    if not dataset_path:
        raise FileNotFoundError(f"Dataset not found in any of the following paths:\n  " + "\n  ".join(possible_paths))

    print(f"Loading dataset from: {dataset_path}")
    df = pd.read_parquet(dataset_path)

    # Define columns
    identifier_cols = ['station_complex_id', 'station_complex', 'transit_timestamp']
    target_col = 'ridership'
    feature_cols = [col for col in df.columns if col not in identifier_cols + [target_col]]

    # Extract components
    X = df[feature_cols].copy()
    y = df[target_col].copy()
    identifiers = df[identifier_cols].copy()

    print(f"Dataset loaded:     {df.shape}")
    print(f"Features:           {len(feature_cols)}")
    print(f"Target column:      {target_col}")
    print(f"Date range:         {df['transit_timestamp'].min()} → {df['transit_timestamp'].max()}")

    return X, y, identifiers, feature_cols


def create_temporal_splits(X, y, identifiers):
    """Create train/test split based on timestamp cutoff (Nov–Dec holdout)."""
    print("\n2. TEMPORAL SPLITTING")
    print("-" * 40)

    timestamps = identifiers['transit_timestamp']
    if not pd.api.types.is_datetime64_any_dtype(timestamps):
        timestamps = pd.to_datetime(timestamps)

    sort_idx = timestamps.argsort()

    X_sorted = X.iloc[sort_idx].reset_index(drop=True)
    y_sorted = y.iloc[sort_idx].reset_index(drop=True)
    ts_sorted = timestamps.iloc[sort_idx].reset_index(drop=True)

    train_cutoff = pd.Timestamp("2024-10-31 23:59:59")
    train_mask = ts_sorted <= train_cutoff

    X_train = X_sorted[train_mask].reset_index(drop=True)
    X_test = X_sorted[~train_mask].reset_index(drop=True)
    y_train = y_sorted[train_mask].reset_index(drop=True)
    y_test = y_sorted[~train_mask].reset_index(drop=True)

    print(f"Train samples:      {len(X_train):,}")
    print(f"Test samples:       {len(X_test):,}")
    print(f"Train target mean:  {y_train.mean():.0f}")
    print(f"Test target mean:   {y_test.mean():.0f}")

    return X_train, X_test, y_train, y_test


In [3]:
# =============================================
# 3. Model Definition, Evaluation, and Comparison
# =============================================

def define_models():
    """Define baseline models."""
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(alpha=1.0, random_state=42),
        'Lasso': Lasso(alpha=1.0, random_state=42, max_iter=2000),
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    }

    if XGBOOST_AVAILABLE:
        models['XGBoost'] = xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)

    if LIGHTGBM_AVAILABLE:
        models['LightGBM'] = lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)

    return models


def evaluate_model(name, model, X_train, X_test, y_train, y_test, skip_cv=False):
    """Evaluate a single model with optional cross-validation."""
    
    # Pipeline (scaling for linear models only)
    if name in ['LinearRegression', 'Ridge', 'Lasso']:
        pipeline = Pipeline([('scaler', StandardScaler()), ('model', model)])
    else:
        pipeline = Pipeline([('model', model)])

    # Cross-validation
    if not skip_cv:
        tscv = TimeSeriesSplit(n_splits=5)
        cv_scores = cross_val_score(
            pipeline, X_train, y_train, cv=tscv,
            scoring='neg_root_mean_squared_error', n_jobs=-1
        )
        cv_rmse = -cv_scores.mean()
        cv_std = cv_scores.std()
    else:
        cv_rmse, cv_std = 0, 0

    # Training
    start_time = datetime.now()
    pipeline.fit(X_train, y_train)
    training_time = (datetime.now() - start_time).total_seconds()

    # Predictions
    train_pred = pipeline.predict(X_train)
    test_pred = pipeline.predict(X_test)

    # Evaluation
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)

    rmse_gap_pct = ((test_rmse - train_rmse) / test_rmse) * 100
    r2_gap = train_r2 - test_r2

    return {
        'model': pipeline,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'rmse_gap_pct': rmse_gap_pct,
        'r2_gap': r2_gap,
        'cv_rmse': cv_rmse,
        'cv_std': cv_std,
        'training_time': training_time
    }


def compare_models(models, X_train, X_test, y_train, y_test):
    """Compare all baseline models."""
    print("\n3. MODEL COMPARISON")
    print("-" * 40)

    results = {}
    comparison_data = []

    for name, model in models.items():
        print(f"Training {name}...")
        result = evaluate_model(name, model, X_train, X_test, y_train, y_test)
        results[name] = result

        comparison_data.append({
            'Model': name,
            'Train_RMSE': result['train_rmse'],
            'Test_RMSE': result['test_rmse'],
            'Train_R2': result['train_r2'],
            'Test_R2': result['test_r2'],
            'RMSE_Gap_%': result['rmse_gap_pct'],
            'R2_Gap': result['r2_gap'],
            'CV_RMSE': result['cv_rmse']
        })

        print(f"  RMSE: {result['test_rmse']:.0f}, R²: {result['test_r2']:.3f}, RMSE Gap: {result['rmse_gap_pct']:.1f}%, R² Gap: {result['r2_gap']:.3f}")

    # Tabular summary
    df_comparison = pd.DataFrame(comparison_data).sort_values('Test_R2', ascending=False)

    print("\nModel Comparison:")
    print(df_comparison.round({
        'Train_RMSE': 0, 'Test_RMSE': 0,
        'Train_R2': 3, 'Test_R2': 3,
        'RMSE_Gap_%': 1, 'R2_Gap': 3, 'CV_RMSE': 0
    }).to_string(index=False))

    return results, df_comparison


In [4]:
# =============================================
# 4. XGBoost Optimization and Evaluation
# =============================================

def optimize_xgboost(X_train, X_test, y_train, y_test):
    """Train and evaluate XGBoost with tuned hyperparameters."""
    print("\n4. XGBOOST OPTIMIZATION")
    print("-" * 40)

    # Tuned hyperparameters
    xgb_params = {
        'n_estimators': 1000,
        'max_depth': 4,
        'reg_alpha': 0.3,
        'reg_lambda': 0.3,
        'min_child_weight': 8,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'learning_rate': 0.05,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1
    }

    print("Model Parameters:")
    for k, v in xgb_params.items():
        print(f"  {k:<20}: {v}")

    # Train model
    print("\nTraining XGBoost...")
    model = xgb.XGBRegressor(**xgb_params)
    start_time = datetime.now()
    model.fit(X_train, y_train)  # Optional: add early_stopping_rounds, eval_set
    training_time = (datetime.now() - start_time).total_seconds()

    # Predict
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    # Metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)
    rmse_gap_pct = ((test_rmse - train_rmse) / test_rmse) * 100
    r2_gap = train_r2 - test_r2

    # Cross-validation
    print("\nCross-validation (TimeSeriesSplit)...")
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = cross_val_score(
        model, X_train, y_train,
        cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1
    )
    cv_rmse_scores = -cv_scores
    cv_rmse_mean = cv_rmse_scores.mean()
    cv_rmse_std = cv_rmse_scores.std()

    # Print summary
    print("\n--- Evaluation Results ---")
    print(f"Train RMSE:      {train_rmse:.2f}")
    print(f"Test RMSE:       {test_rmse:.2f}")
    print(f"Train R²:        {train_r2:.3f}")
    print(f"Test R²:         {test_r2:.3f}")
    print(f"RMSE Gap (%):    {rmse_gap_pct:.1f}%")
    print(f"R² Gap:          {r2_gap:.3f}")
    print(f"Training Time:   {training_time:.1f} sec")

    print("\nCross-Validation RMSE Scores:")
    for i, score in enumerate(cv_rmse_scores, 1):
        print(f"  Fold {i}: {score:.2f}")
    print(f"Mean CV RMSE:    {cv_rmse_mean:.2f}")
    print(f"CV RMSE Std:     {cv_rmse_std:.2f}")

    return {
        'model': model,
        'params': xgb_params,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'rmse_gap_pct': rmse_gap_pct,
        'r2_gap': r2_gap,
        'cv_rmse_mean': cv_rmse_mean,
        'cv_rmse_std': cv_rmse_std,
        'cv_scores': cv_rmse_scores,
        'training_time': training_time,
        'trees_used': xgb_params['n_estimators']
    }


In [5]:
# =============================================
# 5. Feature Importance Analysis
# =============================================

def analyze_feature_importance(model_result, feature_names):
    """Analyze and display feature importances from the optimized XGBoost model."""
    print("\n5. FEATURE IMPORTANCE ANALYSIS")
    print("-" * 40)

    model = model_result['model']

    if not hasattr(model, 'feature_importances_'):
        print("Feature importance not available for this model type.")
        return None

    importances = model.feature_importances_

    if len(importances) != len(feature_names):
        print("Warning: Feature importances do not match number of features.")
        return None

    # Create importance DataFrame
    importance_df = (
        pd.DataFrame({'feature': feature_names, 'importance': importances})
        .sort_values('importance', ascending=False)
        .reset_index(drop=True)
    )

    print(f"Feature Importance Rankings ({len(feature_names)} features):")
    print("-" * 50)
    for i, row in importance_df.iterrows():
        print(f"  {i+1:2d}. {row['feature']:<20}: {row['importance']:.5f}")

    # Category breakdown
    temporal_features = [
        'hour', 'day_of_week', 'month', 'hour_sin', 'hour_cos',
        'dow_sin', 'dow_cos', 'month_sin', 'month_cos',
        'is_rush_hour', 'is_weekend', 'is_holiday'
    ]
    weather_features = [
        'temp', 'humidity', 'wind_speed', 'feels_like',
        'has_snow', 'has_rain', 'is_freezing', 'is_hot', 'temp_category'
    ]
    location_features = ['latitude', 'longitude', 'is_cbd']

    temporal_imp = importance_df[importance_df['feature'].isin(temporal_features)]['importance'].sum()
    weather_imp = importance_df[importance_df['feature'].isin(weather_features)]['importance'].sum()
    location_imp = importance_df[importance_df['feature'].isin(location_features)]['importance'].sum()

    print("\nImportance by Feature Category:")
    print(f"  Temporal Features : {temporal_imp:.5f}")
    print(f"  Weather Features  : {weather_imp:.5f}")
    print(f"  Location Features : {location_imp:.5f}")

    return importance_df


In [6]:
# =============================================
# 6. Business Logic Validation (Predicted Behavior)
# =============================================

def validate_business_logic(model_result, X_test):
    """Validate whether model predictions align with known ridership patterns."""
    print("\n6. BUSINESS LOGIC VALIDATION")
    print("-" * 40)

    model = model_result['model']
    test_pred = model.predict(X_test)

    test_data = X_test.copy()
    test_data['predicted'] = test_pred

    validation_results = {}

    # Rush hour pattern
    if 'is_rush_hour' in test_data.columns:
        rush_avg = test_data[test_data['is_rush_hour'] == 1]['predicted'].mean()
        non_rush_avg = test_data[test_data['is_rush_hour'] == 0]['predicted'].mean()
        rush_factor = rush_avg / non_rush_avg if non_rush_avg > 0 else np.nan
        validation_results['rush_hour_factor'] = rush_factor
        print(f"Rush hour factor:    {rush_factor:.2f}x (expected ~2.36x)")

    # Weekend pattern
    if 'is_weekend' in test_data.columns:
        weekend_avg = test_data[test_data['is_weekend'] == 1]['predicted'].mean()
        weekday_avg = test_data[test_data['is_weekend'] == 0]['predicted'].mean()
        weekend_factor = weekend_avg / weekday_avg if weekday_avg > 0 else np.nan
        validation_results['weekend_factor'] = weekend_factor
        print(f"Weekend factor:      {weekend_factor:.2f}x (expected ~0.63x)")

    # CBD vs non-CBD pattern
    if 'is_cbd' in test_data.columns:
        cbd_avg = test_data[test_data['is_cbd'] == 1]['predicted'].mean()
        non_cbd_avg = test_data[test_data['is_cbd'] == 0]['predicted'].mean()
        cbd_factor = cbd_avg / non_cbd_avg if non_cbd_avg > 0 else np.nan
        validation_results['cbd_factor'] = cbd_factor
        print(f"CBD location factor: {cbd_factor:.2f}x (expected ~2.52x)")

    # Pattern validation status flags
    validation_results['status'] = {
        'rush_hour_valid': 2.0 <= validation_results.get('rush_hour_factor', 0) <= 3.0,
        'weekend_valid':   0.55 <= validation_results.get('weekend_factor', 1) <= 0.75,
        'cbd_valid':       2.0 <= validation_results.get('cbd_factor', 0) <= 3.5
    }

    return validation_results


In [7]:
# =============================================
# 7. Save Final Model and Metadata
# =============================================

def save_final_model(model_result, feature_names, importance_df, validation_results):
    """Save the final trained model along with metadata and supporting files."""
    print("\n7. SAVING FINAL MODEL")
    print("-" * 40)

    output_dir = Path("../data/processed/models")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Save model
    model_file = output_dir / "subway_ridership_model_xgboost_final.joblib"
    joblib.dump(model_result['model'], model_file)

    # Helper for JSON serialization
    def convert_to_json_serializable(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.bool_):  
            return bool(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert_to_json_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_json_serializable(i) for i in obj]
        else:
            return obj

    # Build metadata
    metadata = {
        'model_name': 'XGBoost_Optimized_Final',
        'creation_date': datetime.now().isoformat(),
        'model_file': str(model_file.name),
        'loading_instructions': "Use joblib.load() to load the model",
        'performance': {
            'test_rmse': float(model_result['test_rmse']),
            'test_r2': float(model_result['test_r2']),
            'train_r2': float(model_result.get('train_r2', 0)),
            'rmse_gap_pct': float(model_result['rmse_gap_pct']),
            'r2_gap': float(model_result.get('r2_gap', 0)),
            'cv_rmse_mean': float(model_result['cv_rmse_mean']),
            'cv_rmse_std': float(model_result['cv_rmse_std']),
            'training_time_sec': float(model_result['training_time']),
            'trees_used': int(model_result['trees_used'])
        },
        'cv_scores': convert_to_json_serializable(model_result['cv_scores']),
        'hyperparameters': convert_to_json_serializable(model_result['params']),
        'features': feature_names,
        'business_validation': convert_to_json_serializable(validation_results),
        'feature_importance_top10': (
            importance_df.head(10).to_dict('records') if importance_df is not None else None
        )
    }

    # Save metadata
    metadata_file = output_dir / "model_metadata_xgboost_final.json"
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)

    # Save feature list separately
    features_file = output_dir / "required_features.json"
    with open(features_file, 'w') as f:
        json.dump(feature_names, f, indent=2)

    print(f"Model saved:     {model_file}")
    print(f"Metadata saved:  {metadata_file}")
    print(f"Feature list:    {features_file}")

    return [model_file, metadata_file, features_file]


In [8]:
# =============================================
# 8. Pipeline Orchestration
# =============================================

def main():
    """Run the complete modeling pipeline end-to-end."""
    try:
        print("\n" + "=" * 60)
        print("STARTING PIPELINE")
        print("=" * 60)

        # 1. Data preparation
        X, y, identifiers, feature_names = load_and_prepare_data()
        X_train, X_test, y_train, y_test = create_temporal_splits(X, y, identifiers)

        # 2. Model comparison
        models = define_models()
        baseline_results, baseline_comparison = compare_models(models, X_train, X_test, y_train, y_test)

        # 3. Optimized XGBoost model
        optimized_model_result = optimize_xgboost(X_train, X_test, y_train, y_test)

        # 4. Feature importance analysis
        importance_df = analyze_feature_importance(optimized_model_result, feature_names)

        # 5. Business logic validation
        validation_results = validate_business_logic(optimized_model_result, X_test)

        # 6. Save model + metadata
        saved_files = save_final_model(optimized_model_result, feature_names, importance_df, validation_results)

        # 7. Final summary
        print("\n" + "=" * 60)
        print("PIPELINE COMPLETED")
        print("=" * 60)
        print(f"Final Model:         RMSE        = {optimized_model_result['test_rmse']:.0f}")
        print(f"                     R²          = {optimized_model_result['test_r2']:.3f}")
        print(f"                     RMSE Gap %  = {optimized_model_result['rmse_gap_pct']:.1f}%")
        print(f"                     R² Gap      = {optimized_model_result['r2_gap']:.3f}")
        print(f"Files saved:         {len(saved_files)}")
        print("=" * 60)

        return {
            'baseline_results': baseline_results,
            'optimized_model': optimized_model_result,
            'importance_df': importance_df,
            'validation': validation_results,
            'files': saved_files
        }

    except Exception as e:
        print("\nERROR OCCURRED DURING PIPELINE EXECUTION")
        print("-" * 60)
        print(f"{e}")
        import traceback
        traceback.print_exc()
        return None

# Execute pipeline
if __name__ == "__main__":
    results = main()
else:
    results = main()



STARTING PIPELINE

1. DATA LOADING AND PREPARATION
----------------------------------------
Loading dataset from: ..\data\processed\modeling\subway_ridership_modeling_features.parquet
Dataset loaded:     (1052709, 28)
Features:           24
Target column:      ridership
Date range:         2024-01-01 00:00:00 → 2024-12-31 23:00:00

2. TEMPORAL SPLITTING
----------------------------------------
Train samples:      876,990
Test samples:       175,719
Train target mean:  637
Test target mean:   674

3. MODEL COMPARISON
----------------------------------------
Training LinearRegression...
  RMSE: 1054, R²: 0.206, RMSE Gap: 5.9%, R² Gap: 0.021
Training Ridge...
  RMSE: 1054, R²: 0.206, RMSE Gap: 5.9%, R² Gap: 0.021
Training Lasso...
  RMSE: 1048, R²: 0.215, RMSE Gap: 5.3%, R² Gap: 0.012
Training RandomForest...
  RMSE: 316, R²: 0.929, RMSE Gap: 83.7%, R² Gap: 0.069
Training XGBoost...
  RMSE: 354, R²: 0.910, RMSE Gap: 43.9%, R² Gap: 0.059
Training LightGBM...
  RMSE: 375, R²: 0.899, RMSE G